In [1]:
import sys
sys.path.append('../02_dataset_generation')

In [2]:
import json

In [3]:
from schema_to_sqlalchemy import tables_to_sqlalchemy_file

In [4]:
with open('../db_schemas.json', 'r') as f:
    schemas = json.load(f)

In [5]:
tables = schemas[0].get('tables')

In [6]:
await tables_to_sqlalchemy_file(tables, 'output_sqlalchemy_schema.py')

In [8]:
from output_sqlalchemy_schema import create

In [9]:
engine = create()

In [11]:
import pandas as pd

In [13]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", engine)

,name
0,Customers
1,Flights
2,Bookings


In [14]:
pd.read_sql("SELECT * FROM Customers", engine)

,customer_id,first_name,last_name,email,phone_number,address


In [18]:
from create_data import populate_data

In [19]:
await populate_data('output_sqlalchemy_schema.py', 'populate_data.py')

In [ ]:
from populate_data import 